Important, at the end check that the code is clean, that all the debug cells are removed (checking shape when not required and so on), follow the numbered structure of the exam with markdown cells, do generalized solutions

In [2]:
import sklearn
sklearn_version = sklearn.__version__

print(sklearn_version)

0.24.2


In [ ]:
#At the beginning of every exam import those and then check if something is missing or not useful and delete it

from IPython.display import Image
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split


%matplotlib inline

random_state = 42

In [ ]:
#When we have a csv file without header and column names we usually have to do this, but the second method works and is easier

data_file = 'name.csv'
delimiter = ',' #Check for the delimiter in the file, it could also be ;
X = np.loadtxt(data_file, delimiter = delimiter)

df = pd.DataFrame(X)

#Or if we have problems with conversion, also this

df = pd.read_csv('name.csv', header = None)
#In this particular case in order to access the columns of the dataframe we need to do df[num] where num is the 'label' of the
#column we want to access, remember not to put it in apices, so not df['num']

In [ ]:
#Remember to always store the modifications to dataset, e.g.
df.shape
df.dropna()
df.shape
#Won't result in any change because we didn't save the new value of df without Nan values, on the other hand
df.shape
df = df.dropna()
df.shape
#Result in a change now because we have stored it


## Data Exploration

In [ ]:
df.head()
df.shape

# Checking which columns contain numerical data and which contain categorical data.
df.dtypes

#Structure of numeric data
df_num = df.drop(class_label,axis=1) # axis= 1 drops the column
df_num.describe()

#If we want to have a description also of non numeric data we need to explicitly say
cat_attributes = df.dtypes.loc[df.dtypes=='object'].index.values 
df[cat_attributes].describe()

#Histograms
df_num.hist(figsize=(10,10));

#Boxplot
df.boxplot()
#Usually boxplot comments are on the range of columns, if much smaller or similar

#Pairplot
sns.pairplot(df, hue = class_label) #if present
#Comments on pairplots are usually looking at the couple of attributes to focus on that visually generate the best data 
#arrangement to be clustered e.g. The pairplots show that the two most interesting columns are `1` and `2`, their pairplot shows
#evident clusters. The pairplots of `0` and `3` show that those columns are uniformly distributed and do not show any pattern.


#Scatterplot of itneresting columns of the pairplot
interesting_columns = [0,1]
plt.scatter(X[:,interesting_columns[0]], X[:,interesting_columns[1]]
            , c='white'          # color filling the data markers
            , edgecolors='black' # edge color for data markers
            , marker='o'         # data marker shape, e.g. triangles (v<>^), square (s), star (*), ...
            , s=50)              # data marker size
plt.grid()  
plt.show()

#OR
chosen_dim = [0,1]
plot_data = pd.DataFrame(X_red)
plot_data[class_label] =  y
sns.scatterplot(x = chosen_dim[0],y = chosen_dim[1],data=plot_data, hue = class_label);


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = random_state)

## Data Preprocessing

In [ ]:
#Removing of NaNs values
df = df.dropna(axis = 0)

#Data types
print(df.dtypes)


### From ordinal/categorical to numeric data

- the **ordinal transformer** generates a mapping from strings to numbers according to the lexicographic sorting of the strings; in this particular case, the strings indicate numeric subranges, and ranges with one digit constitute exceptions
        '5-9' happens to be after '20-25'
    - it is necessary to transform '5-9' into '05-09', and the same for other similar cases
    - a way to do this is to prepare dictionaries for the translation and use the `.map` function
    
Vedi lab 04 Data Preprocessing

In [ ]:
non_numeric_features = df.dtypes.loc[df.dtypes == 'object'].index.values
print("The non-numeric features are:")
print(non_numeric_features)

numeric_features = list(set(df.dtypes.index.values)-set(non_numeric_features))
print("The numeric features are:")
print(numeric_features)

ordinal_features =['age', 'tumor-size','inv-nodes'] #This is an example, ordinal features are chosen by hand
print("The ordinal features are:")
print(ordinal_features)

categorical_features = list(set(non_numeric_features) - set(ordinal_features) - set(['Class']))
print("The categorical features are:")
print(categorical_features)

##### If I have to transform only ordinal to numeric

In [ ]:
focus = [2] # list of ordinal attributes
transf_dtype = np.int32 # type to be used when converting


ordinal_transformer = OrdinalEncoder(dtype = transf_dtype) # we assume the values are encoded so that lexicographic order = intended order
df_2 = df.copy()
df_2[focus] = ordinal_transformer.fit_transform(df[focus])
df_2.head() #just to check

##### If I have to transform both ordinal and categorical to numeric at the same time

In [ ]:
#Prepare transformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

transf_dtype = np.int32

categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse = False, dtype = transf_dtype)
ordinal_transformer = OrdinalEncoder(dtype = transf_dtype)
preprocessor = ColumnTransformer(
    transformers = [('cat', categorical_transformer, categorical_features),
                    ('ord', ordinal_transformer, ordinal_features)
                   ],
                    remainder = 'passthrough'
    )

X = df.drop(['Class'], axis = 1)
y = df['Class']

preprocessor.fit(X)
print(preprocessor.named_transformers_)

X_p = preprocessor.fit_transform(X)
df_p = pd.DataFrame(X_p) #just to check
df_p.head()
X_train, X_test, y_train, y_test = train_test_split(X_p,y, random_state = random_state)

##### Don't change data type of class label

In classification it does not matter what the class actually represents, the learning algorithm treats every class as categorical anyway. In other words whether the names of the classes are strings, characters or numbers does not change anything to the model.

It would be a bad idea to use one hot encoding because this would make the problem multi-label. This would make the problem much more complex for the model and would very likely lead to lower performance, or it would require much more data in order to reach the same performance as regular classification. This is because there are much more combinations possible in the multi-label problem, and in this case this higher level of complexity is pointless since there can be only one class.

### Feature selection

In [ ]:
from sklearn.feature_selection import SelectKBest, SelectPercentile 
from sklearn.feature_selection import mutual_info_classif
from functools import partial

# k = num, num is the number of the best features that will be selected (e.g. if I start from 4 features k must be <4)

kbest = SelectKBest(score_func=partial(mutual_info_classif,random_state=random_state), k=num) 

fit = kbest.fit(X,y)
X_reducted = fit.transform(X)
X_reducted.shape

#Other selection methods are in the lab about feature selection

Features ranking with RFE (Recursive Feature elimination)

In [ ]:
from sklearn.feature_selection import RFE
# feature extraction
rfe = RFE(estimator, n_features_to_select=5)
fit = rfe.fit(X, y)
X_reducted = fit.transform(X)

print("Feature Ranking: %s", fit.ranking_)

Feature selection unsupervised with PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
fit = pca.fit(X)

print("Explained Variance:", fit.explained_variance_ratio_)
X_reducted = fit.transform(X)

### Rescaling data

In [ ]:
scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)

## Classification
Hyperparameter tuning with cross-validation (GridSearchCV)

In [ ]:
classes = df[target_name].unique()
classes.sort()
print(classes)

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

model_lbls = [
#              'dt', 
#              'nb', 
#              'lp', 
#              'svc', 
             'knn',
            ]

# Set the parameters by cross-validation
tuned_param_dt = [{'max_depth': [range(1,20)]}]
tuned_param_nb = [{'var_smoothing': [10, 1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-07, 1e-8, 1e-9, 1e-10]}]
tuned_param_lp = [{'early_stopping': [True]}]
tuned_param_svc = [{'kernel': ['rbf'], 
                    'gamma': [1e-3, 1e-4],
                    'C': [1, 10, 100, 1000],
                    },
                    {'kernel': ['linear'],
                     'C': [1, 10, 100, 1000],                     
                    },
                   ]
tuned_param_knn =[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}]

models = {
    'dt': {'name': 'Decision Tree       ',
           'estimator': DecisionTreeClassifier(), 
           'param': tuned_param_dt,
          },
    'nb': {'name': 'Gaussian Naive Bayes',
           'estimator': GaussianNB(),
           'param': tuned_param_nb
          },
    'lp': {'name': 'Linear Perceptron   ',
           'estimator': Perceptron(),
           'param': tuned_param_lp,
          },
    'svc':{'name': 'Support Vector      ',
           'estimator': SVC(), 
           'param': tuned_param_svc
          },
    'knn':{'name': 'K Nearest Neighbor ',
           'estimator': KNeighborsClassifier(),
           'param': tuned_param_knn
        
    }
}

scores = ['precision', #'recall', recall_macro, accuracy]

In [ ]:
#Scores and classification report

def print_results(model):
    print("Best parameters set found on train set:")
    print()
    # if best is linear there is no gamma parameter
    print(model.best_params_)
    print()
    print("Grid scores on train set:")
    print()
    means = model.cv_results_['mean_test_score']
    stds = model.cv_results_['std_test_score']
    params = model.cv_results_['params']
    for mean, std, params_tuple in zip(means, stds, params):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params_tuple))
    print()
    print("Detailed classification report for the best parameter set:")
    print()
    print("The model is trained on the full train set.")
    print("The scores are computed on the full test set.")
    print()
    y_true, y_pred = y_test, model.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

In [ ]:
results_short = {}

for score in scores:
    print('='*40)
    print("# Tuning hyper-parameters for %s" % score)
    print()

    #'%s_macro' % score ## is a string formatting expression
    # the parameter after % is substituted in the string placeholder %s
    for m in model_lbls:
        print('-'*40)
        print("Trying model {}".format(models[m]['name']))
        clf = GridSearchCV(models[m]['estimator'], models[m]['param'], cv=5,
                           scoring='%s_macro' % score, 
                           iid = False, 
                           return_train_score = False,
                           n_jobs = 2, # this allows using multi-cores
                           )
        clf.fit(X_train, y_train)
        print_results(clf)
        results_short[m] = clf.best_score_
    print("Summary of results for {}".format(score))
    print("Estimator")
    for m in results_short.keys():
        print("{}\t - score: {:4.2}%".format(models[m]['name'], results_short[m]))

##### Doing separatedly model 1 and model 2

In [ ]:
# Setting the parameters to be explored by GridSearchCV for the decision tree model.
tuned_param_dt = [{"max_depth": list(range(1, 16))}]

# Setting the parameters to be explored by GridSearchCV for the k nearest neighbor model.
tuned_param_knn =[{"n_neighbors": list(range(1, 16))}]

# Defining the models to be fitted.
models = {
    "Model1": {"name": "Decision Tree",      "estimator": DecisionTreeClassifier(), "param": tuned_param_dt},
    "Model2": {"name": "K Nearest Neighbor", "estimator": KNeighborsClassifier(),   "param": tuned_param_knn}
}

# Defining the list of scores to be explored.
score = "recall_macro"

In [ ]:
# Defining the function which takes as input the fitted model and returns the classification report.
def print_results(name, model, y_test, y_pred):
    
    # Printing the type of model.
    print("Model name: {}.".format(name))
    
    # Printing the grid scores obtained during training.
    print("\nGrid scores:")
    
    # Iterating over the grid rows.
    for mean, std, params in zip(model.cv_results_["mean_test_score"], model.cv_results_["std_test_score"], model.cv_results_["params"]):
        
        # Printing one row of the grid.
        print("{:.3f} (+/- {:.3f}) for {}".format(mean, std * 2, params))
    
    # Printing the best parameters set.
    print("\nBest parameters set: {}.".format(model.best_params_))
    
    # Printing the detailed classification report.
    print("\nDetailed classification report:\n{}".format(classification_report(y_test, y_pred)))

In [ ]:
#Hyperparameter tuning for the first model 
# Activating the grid search.
clf = GridSearchCV(models["Model1"]["estimator"], models["Model1"]["param"], scoring = score, cv = 5)

# Fitting the model.
clf.fit(X_train, y_train);

##### Best hyperparameters if report is not requested

In [ ]:
print("The best found hyperparameters are: {}".format(clf.best_params_))

##### Classification report

In [ ]:
#Classification report for the first model
# Computing predictions with the test set.
y_pred = clf.predict(X_test)

# Printing the results.
print_results(models["Model1"]["name"], clf, y_test, y_pred)

##### Confusion matrix and accuracy score

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix

y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("The accuracy on test set is {:.2f}%".format(acc*100))
print("Confusion matrix:\n{}".format(confusion_matrix(y_test, y_pred)))

#Different confusion matrix normalizing for each class
plot_confusion_matrix(clf, X_test, y_test, normalize='pred');

## Clustering

When have to decide between K-means and DBSCAN see:
1. If in the pairplot of choosen data there are convex distributions or outliers (if yes select DBSCAN)
2. If clusters vary a lot in densities (if yes select K-means)

### K-means
In k-means the best number of cluster (the only hyperparameter) can be selected via elbow method by looking at the graph of distortions and silhouette scores of estimators fitted with different values of k.
If this is not enough a visual inspection of clustered data can be done

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

n_cluster_range = range(2,10)
distortions = []
silhouette_scores = []

for n_clust in n_cluster_range:

    km = KMeans(n_clusters = n_clust,random_state = random_state)
    
    
    y_km = km.fit_predict(X)
    distortion = km.inertia_
    silhouette = silhouette_score(X,y_km)
    
    distortions.append(distortion)
    silhouette_scores.append(silhouette)
    
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Number of clusters')
ax1.set_ylabel('Inertia', color=color)
ax1.plot(n_cluster_range, distortions, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  

color = 'tab:blue'
ax2.set_ylabel('Silhouette scores', color=color)  
ax2.plot(n_cluster_range, silhouette_scores, color=color)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylim(0,1) 

fig.tight_layout() 
plt.show()

In [ ]:
good_k = #k chosen before via elbow method
km = KMeans(n_clusters=good_k, 
            init='k-means++', 
            n_init=10, 
            max_iter=300,
            tol=1e-04,
            random_state=random_state)
y_km = km.fit_predict(X)

#Silhouette score of the best parameter choice
print(silhouette_score(X,y_km))

### DBSCAN
With DBSCAN if I don't have supervised information on class labels I can tune the hyperparameters (n_clusters, epsilon and minPoints) via GridSearch, plotting then a report and manually choosing the best one with a tradeoff between silhouette score and unclustered data

For a good starting approximation of min_points and eps we do as follows

In [ ]:
#First of all if we have a dataframe we transform it into a numpy array just for the sake of the following two cells
X = df.to_numpy()

In [ ]:
#For min_points we look around twice the number of attributes of the dataset (X.shape[1])
min_points=2*X.shape[1] 

In [ ]:
#For eps we look at the elbow curve of the following graph and we range in its proximity (in this case it was 0.2)
k_distances = []
for i in range(0, X.shape[0]):
    k_point_distances = []
    for j in range(0, X.shape[0]):
        if i!=j:
            dist = np.sqrt(sum((X[i,:]-X[j,:])**2))
            k_point_distances.append(dist)
    k_point_distances.sort()
    k_distances.append(k_point_distances[min_points-1])
    
k_distances.sort(reverse=True)
plt.plot(range(0,len(k_distances)), k_distances)
plt.show()

ATTENTION, the last cell is the ParameterGrid in case of already choosen number of clusters (e.g. in case of supervised label), the one that follows is the more general one in which number of clusters is an hyperparameter free to be tuned

ATTENTION if the thresholds are set too high (the sil one) or too low (the unc one) no result will be displayed

In [ ]:
#Version with tuning also of the number of clusters hyperparameter, not previously fixed

from sklearn.model_selection import ParameterGrid
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, silhouette_samples

grid = {'eps':list(np.arange(0.1 ,0.3, 0.01)), 'min_samples':list(range(min_points-4,min_points+4,1))}
params = list(ParameterGrid(grid))
sil_thr = 0.56  # visualize results only for combinations with silhouette above the threshold
unc_thr = 10    # visualize results only for combinations with uncl_ratio below the threshold
columns = ["eps", "min_samples", "n_clusters", "silhouette", "unclust%"]
result = pd.DataFrame(columns=columns)



for i in params:
    grid_est = DBSCAN(eps=i['eps'], min_samples=i['min_samples'])
    yge = grid_est.fit_predict(X)
    labels = np.unique(yge)
    if -1 in labels:
        n=1
    else:
        n=0
    uncl_ratio = np.count_nonzero(yge==-1)/len(yge)*100
    n_clusters = np.count_nonzero(labels>-1)
    y_no_noise = yge[yge!=-1]
    
    if len(labels)-n < 2:
        sil_score=-1
        sdi=-1
    else:
        sil_score = silhouette_score(X[yge!=-1], y_no_noise)
        
    if sil_score>sil_thr and uncl_ratio < unc_thr:
        result = result.append({ 'eps': i["eps"],
                                'min_samples':i["min_samples"],
                                'n_clusters':n_clusters,
                                'silhouette':sil_score,
                                'unclust%':uncl_ratio}, ignore_index=True)
        
print(result)

In [ ]:
#Version with an already fixed number of clusters (i.e. required_clusters) equal to the number of classes in the supervised 
#column

from sklearn.model_selection import ParameterGrid
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, silhouette_samples

grid = {'eps':list(np.arange(0.1 ,0.3, 0.01)), 'min_samples':list(range(min_points-4,min_points+4,1))}
params = list(ParameterGrid(grid))
sil_thr = 0.56  # visualize results only for combinations with silhouette above the threshold
unc_thr = 10    # visualize results only for combinations with uncl_ratio below the threshold
columns = ["eps", "min_samples", "n_clusters", "silhouette", "unclust%"]
result = pd.DataFrame(columns=columns)
required_clusters = len(np.unique(y))


for i in params:
    grid_est = DBSCAN(eps=i['eps'], min_samples=i['min_samples'])
    yge = grid_est.fit_predict(X)
    labels = np.unique(yge)
    if -1 in labels:
        n=1
    else:
        n=0
    uncl_ratio = np.count_nonzero(yge==-1)/len(yge)*100
    n_clusters = np.count_nonzero(labels>-1)
    y_no_noise = yge[yge!=-1]
    
    if len(labels)-n < 2:
        sil_score=-1
        sdi=-1
    else:
        sil_score = silhouette_score(X[yge!=-1], y_no_noise)
        
    if n_clusters==required_clusters and sil_score>sil_thr and uncl_ratio < unc_thr:
        result = result.append({ 'eps': i["eps"],
                                'min_samples':i["min_samples"],
                                'n_clusters':n_clusters,
                                'silhouette':sil_score,
                                'unclust%':uncl_ratio}, ignore_index=True)
        
print(result)

Manually choose the best set of hyperparameters looking at: silhouette score, uncluster % and number of clusters

In [ ]:
chosen_hyper_db = {'eps':0.18, 'min_samples':4}
dbscan = DBSCAN(eps = chosen_hyper_db['eps'], min_samples=chosen_hyper_db['min_samples'])
y_db = dbscan.fit_predict(X)

### Showing a pairplot of the clustered data
This can be done to visually check the best number of clusters and not only rely on parametergrid for DBSCAN or elbow method for K means

In [ ]:
df['cluster'] = y_km # or y_db

sns.pairplot(df, hue = 'cluster');

#Or if I want to show only interesting columns 

sns.pairplot(df[int_cols + ['cluster']], vars = df[int_cols], hue = 'cluster');


### Assessing cluster quality via silhouette plots

In [ ]:
# Use the plot_silhouette function in the plot_silhouette.py in useful

from sklearn.metrics import silhouette_samples

cluster_labels = np.unique(y_km)
n_clusters = cluster_labels.shape[0] # it is the number of rows
# Compute the Silhouette Coefficient for each sample, with the euclidean metric
silhouette_score_samples = silhouette_samples(X, y_km, metric='euclidean')
plt.title('Silhouette score for samples with {} clusters'.format(good_k))
plot_silhouette(silhouette_score_samples, y_km)

### Using y supervised column as gold standard
In order to correcly have an estimate of the accuracy, clusters (as well as noise points) will be mapped to the highest class they represent
Then I will compute accuracy score between prediction and gold standard and confusion matrix

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

def remap(y_true, y_pred):
    y_mapped = y_pred.copy()
    for lab in np.unique(y_pred):
        true_l, count = np.unique(y_true[y_pred==lab], return_counts=True)
        y_mapped[y_pred==lab] = true_l[np.argmax(count)]
    return y_mapped 

y_db_rem = remap(y, y_db) #or y_km
print("The accuracy is: {:.2f}%".format(accuracy_score(y,y_db_rem)*100))
print("The confusion matrix:")
print(confusion_matrix(y,y_db_rem))

### A sorted list of the discovered clusters for decreasing sizes
I list the number of data points inside each cluster, in this case in decreasing order

In [ ]:
labels = pd.DataFrame(y_km)
counts = pd.DataFrame(labels.value_counts(), columns=['Count']).reset_index()
counts = counts.rename(columns = {0: 'Cluster'})
counts

### If asked to show  the plot of the global inertia and silhouette index for the parameters under examination in case of DBSCAN see ml_lab_exam_simulation_dbscan